# CARP

In [1]:
%cd ~/repo/protein-transfer

/home/t-fli/repo/protein-transfer


In [2]:
%load_ext blackcellmagic

In [3]:
from sequence_models.pretrained import load_model_and_alphabet

In [4]:
model, collater = load_model_and_alphabet('carp_640M')

In [5]:
seqs = [['MDREQ'], ['MGTRRLLP']]
x = collater(seqs)[0]  # (n, max_len)
x
# rep = model(x)  # (n, max_len, d_model)

tensor([[10,  2, 14,  3, 13, 27, 27, 27],
        [10,  5, 16, 14, 14,  9,  9, 12]])

In [24]:
model(x)

tensor([[[ -1.0284,  -8.3767,   7.8792,  ...,   7.2597,  -1.1452,  -0.8462],
         [  4.6713,  15.8570,   1.1478,  ...,   0.7353, -11.8218,   3.7328],
         [  8.4552,  17.9306,   0.5176,  ...,   3.0368,  -8.7715,   0.5799],
         ...,
         [ 15.8914,  32.4650,   1.8229,  ...,  -1.0857,  -3.7955,   3.6075],
         [ 22.5673,  29.2003,  -4.6042,  ...,  -1.7818,  -3.7571,   9.1396],
         [ 15.0128,  30.3642,   0.1932,  ...,  -5.9510,   7.3621,   8.7913]],

        [[ -0.3449, -18.9636,   1.3514,  ...,   3.3504,  -1.8757,   2.4882],
         [ 10.1161,  24.4867,  20.2836,  ...,  18.4754,  -9.3882,   4.8050],
         [ 18.6814,  -3.9989,  -4.1916,  ...,  -2.4737,  -3.2648,   4.1230],
         ...,
         [ 13.9370,  16.6294,  10.2726,  ...,  -6.0374, -15.9087,  -4.9570],
         [ 12.7606,  14.4539,   7.7045,  ...,  -7.9381, -13.7480,  -6.9303],
         [  9.8845,  -6.2927,   0.5083,  ...,   1.9903,  -7.2040,   3.5530]]],
       grad_fn=<AddBackward0>)

In [42]:
model.model.embedder.layers[0]

ByteNetBlock(
  (conv): MaskedConv1d(1280, 1280, kernel_size=(5,), stride=(1,), padding=(2,))
  (sequence1): Sequential(
    (0): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    (1): GELU()
    (2): PositionFeedForward(
      (conv): Conv1d(1280, 1280, kernel_size=(1,), stride=(1,))
    )
    (3): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    (4): GELU()
  )
  (sequence2): Sequential(
    (0): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    (1): GELU()
    (2): PositionFeedForward(
      (conv): Conv1d(1280, 1280, kernel_size=(1,), stride=(1,))
    )
  )
)

In [28]:
model.model.embedder.layers[0].register_forward_hook(lambda m, input, output: output)

In [29]:
model(x)

tensor([[[-0.5807, -3.8715,  0.3539,  ...,  0.5167,  0.7709,  0.3575],
         [ 0.2141,  0.5058, -0.6233,  ...,  0.1069,  0.3125,  0.1335],
         [-0.4885,  1.5820, -0.7261,  ...,  0.6289, -0.1414,  0.0143],
         ...,
         [-0.1115,  1.1351, -0.5492,  ...,  0.9252, -0.5605, -0.8744],
         [-0.5841,  0.9891, -0.4321,  ...,  0.6899, -0.4982, -0.8168],
         [ 0.3778,  1.1565, -0.6100,  ...,  0.1163, -0.5558,  0.1095]],

        [[ 0.0379, -4.0834, -0.4890,  ...,  0.3069,  0.8966,  0.5201],
         [ 0.0405,  0.7682,  0.2539,  ...,  0.1911, -0.5259,  0.1394],
         [ 0.2669, -3.0594, -0.4116,  ...,  0.1007,  0.1165,  0.1414],
         ...,
         [-0.0587,  0.2477,  0.8138,  ...,  0.4770, -0.2852,  0.0514],
         [-0.3768,  0.9766, -0.2094,  ...,  0.1544,  0.0924, -0.0869],
         [-0.5286, -2.1833, -1.1581,  ..., -0.2358, -0.1124,  0.3132]]],
       grad_fn=<AddBackward0>)


tensor([[[ -1.0284,  -8.3767,   7.8792,  ...,   7.2597,  -1.1452,  -0.8462],
         [  4.6713,  15.8570,   1.1478,  ...,   0.7353, -11.8218,   3.7328],
         [  8.4552,  17.9306,   0.5176,  ...,   3.0368,  -8.7715,   0.5799],
         ...,
         [ 15.8914,  32.4650,   1.8229,  ...,  -1.0857,  -3.7955,   3.6075],
         [ 22.5673,  29.2003,  -4.6042,  ...,  -1.7818,  -3.7571,   9.1396],
         [ 15.0128,  30.3642,   0.1932,  ...,  -5.9510,   7.3621,   8.7913]],

        [[ -0.3449, -18.9636,   1.3514,  ...,   3.3504,  -1.8757,   2.4882],
         [ 10.1161,  24.4867,  20.2836,  ...,  18.4754,  -9.3882,   4.8050],
         [ 18.6814,  -3.9989,  -4.1916,  ...,  -2.4737,  -3.2648,   4.1230],
         ...,
         [ 13.9370,  16.6294,  10.2726,  ...,  -6.0374, -15.9087,  -4.9570],
         [ 12.7606,  14.4539,   7.7045,  ...,  -7.9381, -13.7480,  -6.9303],
         [  9.8845,  -6.2927,   0.5083,  ...,   1.9903,  -7.2040,   3.5530]]],
       grad_fn=<AddBackward0>)

In [35]:
rep = model(x)

tensor([[[-0.5807, -3.8715,  0.3539,  ...,  0.5167,  0.7709,  0.3575],
         [ 0.2141,  0.5058, -0.6233,  ...,  0.1069,  0.3125,  0.1335],
         [-0.4885,  1.5820, -0.7261,  ...,  0.6289, -0.1414,  0.0143],
         ...,
         [-0.1115,  1.1351, -0.5492,  ...,  0.9252, -0.5605, -0.8744],
         [-0.5841,  0.9891, -0.4321,  ...,  0.6899, -0.4982, -0.8168],
         [ 0.3778,  1.1565, -0.6100,  ...,  0.1163, -0.5558,  0.1095]],

        [[ 0.0379, -4.0834, -0.4890,  ...,  0.3069,  0.8966,  0.5201],
         [ 0.0405,  0.7682,  0.2539,  ...,  0.1911, -0.5259,  0.1394],
         [ 0.2669, -3.0594, -0.4116,  ...,  0.1007,  0.1165,  0.1414],
         ...,
         [-0.0587,  0.2477,  0.8138,  ...,  0.4770, -0.2852,  0.0514],
         [-0.3768,  0.9766, -0.2094,  ...,  0.1544,  0.0924, -0.0869],
         [-0.5286, -2.1833, -1.1581,  ..., -0.2358, -0.1124,  0.3132]]],
       grad_fn=<AddBackward0>)


In [11]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [12]:
model.model.embedder.layers[0].register_forward_hook(get_activation("layer0"))

In [13]:
rep = model(x)

In [14]:
activation["layer0"]

tensor([[[-0.5807, -3.8715,  0.3539,  ...,  0.5167,  0.7709,  0.3575],
         [ 0.2141,  0.5058, -0.6233,  ...,  0.1069,  0.3125,  0.1335],
         [-0.4885,  1.5820, -0.7261,  ...,  0.6289, -0.1414,  0.0143],
         ...,
         [-0.1115,  1.1351, -0.5492,  ...,  0.9252, -0.5605, -0.8744],
         [-0.5841,  0.9891, -0.4321,  ...,  0.6899, -0.4982, -0.8168],
         [ 0.3778,  1.1565, -0.6100,  ...,  0.1163, -0.5558,  0.1095]],

        [[ 0.0379, -4.0834, -0.4890,  ...,  0.3069,  0.8966,  0.5201],
         [ 0.0405,  0.7682,  0.2539,  ...,  0.1911, -0.5259,  0.1394],
         [ 0.2669, -3.0594, -0.4116,  ...,  0.1007,  0.1165,  0.1414],
         ...,
         [-0.0587,  0.2477,  0.8138,  ...,  0.4770, -0.2852,  0.0514],
         [-0.3768,  0.9766, -0.2094,  ...,  0.1544,  0.0924, -0.0869],
         [-0.5286, -2.1833, -1.1581,  ..., -0.2358, -0.1124,  0.3132]]])

In [15]:
activation["layer0"].shape

torch.Size([2, 8, 1280])

In [16]:
model.model.embedder.layers[30].register_forward_hook(get_activation("layer30"))

In [18]:
rep = model(x)

In [19]:
activation["layer30"], activation["layer30"].shape

(tensor([[[ 3.1358, -1.4431, -1.5922,  ...,  1.7289,  3.0116, -0.9578],
          [ 3.5927, 11.8457, -3.2833,  ...,  3.9837,  1.6558, -2.8215],
          [ 4.8968,  8.8671, -2.4634,  ...,  2.5336,  1.5940, -0.4302],
          ...,
          [10.9340, 10.5128, -1.2239,  ...,  2.5296,  0.4757,  5.3715],
          [10.7995,  7.4243, -3.9322,  ..., -0.1993,  1.3902,  4.4353],
          [10.1016, 10.2742, -4.0287,  ...,  0.4354,  4.8301,  6.8638]],
 
         [[ 3.6099, -8.4282, -2.6635,  ...,  1.7940,  4.2331, -0.1905],
          [ 3.9157, 10.3297,  0.8302,  ...,  4.3193,  3.1261,  0.9150],
          [ 4.3559, -6.4370, -3.3311,  ...,  2.5390,  3.0527,  1.4069],
          ...,
          [ 3.3398,  9.6685,  0.7930,  ...,  3.5631,  0.7518, -0.7545],
          [ 0.7680,  9.1693, -0.7720,  ...,  3.2202,  1.5689, -3.2954],
          [ 1.5965, -2.5849, -2.3010,  ...,  3.4427,  2.7374, -2.5098]]]),
 torch.Size([2, 8, 1280]))

In [42]:
"""Add encoding classes with class methods"""

from __future__ import annotations

from abc import ABC, abstractmethod
from collections.abc import Iterable, Sequence

import numpy as np
from tqdm import tqdm

import torch
from sequence_models.pretrained import load_model_and_alphabet

from scr.params.emb import TRANSFORMER_INFO, TRANSFORMER_MAX_SEQ_LEN, CARP_INFO
from scr.params.sys import DEVICE


class AbstractEncoder(ABC):
    """
    An abstract encoder class to fill in for different kinds of encoders

    All encoders will have an "encode" function
    """

    def __init__(self, encoder_name: str, embed_layer: int):

        """
        Args:
        - encoder_name: str, the name of the encoder
        - embed_layer: int, the layer number of the embedding
        """

        self._encoder_name = encoder_name
        self._embed_layer = embed_layer

    def encode(
        self,
        mut_seqs: Sequence[str] | str,
        batch_size: int = 0,
        flatten_emb: bool | str = False,
        mut_names: Sequence[str] | str | None = None,
    ) -> Iterable[np.ndarray]:
        """
        A function takes a list of sequences to yield a batch of encoded elements

        Args:
        - mut_seqs: list of str or str, mutant sequences of the same length
        - batch_size: int, set to 0 to encode all in a single batch
        - flatten_emb: bool or str, if and how (one of ["max", "mean"]) to flatten the embedding
        - mut_names: list of str or str or None, mutant names
        """

        if isinstance(mut_seqs, str):
            mut_seqs = [mut_seqs]

        # If the batch size is 0, then encode all at once in a single batch
        if batch_size == 0:
            yield self._encode_batch(
                mut_seqs=mut_seqs, flatten_emb=flatten_emb, mut_names=mut_names
            )

        # Otherwise, yield chunks of encoded sequence
        else:

            for i in tqdm(range(0, len(mut_seqs), batch_size)):

                # figure out what mut_names to feed in
                if mut_names is None:
                    mut_name_batch = mut_names
                else:
                    mut_name_batch = mut_names[i : i + batch_size]

                yield self._encode_batch(
                    mut_seqs=mut_seqs[i : i + batch_size],
                    flatten_emb=flatten_emb,
                    mut_names=mut_name_batch,
                )

    def flatten_encode(
        self, encoded_mut_seqs: np.ndarray, flatten_emb: bool | str
    ) -> np.ndarray:
        """
        Flatten the embedding or just return the encoded mutants.

        Args:
        - encoded_mut_seqs: np.ndarray, shape [batch_size, seq_len, embed_dim]
        - flatten_emb: bool or str, if and how (one of ["max", "mean"]) to flatten the embedding
            - True -> shape [batch_size, seq_len * embed_dim]
            - "max" or "mean" -> shape [batch_size, embed_dim]
            - False or everything else -> [batch_size, seq_len, embed_dim]

        Returns:
        - np.ndarray, shape depends on flatten_emb parameter
        """
        assert encoded_mut_seqs.shape[2] == self._embed_dim, "Wrong embed dim"

        if flatten_emb is True:
            # shape [batch_size, seq_len * embed_dim]
            return encoded_mut_seqs.reshape(encoded_mut_seqs.shape[0], -1)

        elif isinstance(flatten_emb, str):
            if flatten_emb == "mean":
                # [batch_size, embed_dim]
                return encoded_mut_seqs.mean(axis=1)
            elif flatten_emb == "max":
                # [batch_size, embed_dim]
                return encoded_mut_seqs.max(axis=1)

        else:
            print("No embedding flattening")
            # [batch_size, seq_len, embed_dim]
            return encoded_mut_seqs

    @abstractmethod
    def _encode_batch(
        mut_seqs: Sequence[str] | str,
        flatten_emb: bool | str,
        mut_names: Sequence[str] | str | None = None,
    ) -> np.ndarray:
        """
        Encode a single batch of mut_seqs
        """
        pass

    @property
    def embed_dim(self) -> int:
        """The dim of the embedding"""
        return self._embed_dim

    @property
    def embed_layer(self) -> int:
        """The layer nubmer of the embedding"""
        return self._embed_layer

    @property
    def encoder_name(self) -> str:
        """The name of the encoding method"""
        return self._encoder_name


class ESMEncoder(AbstractEncoder):
    """
    Build an ESM encoder
    """

    def __init__(
        self,
        encoder_name: str,
        embed_layer: int,
        iftrimCLS: bool = True,
        iftrimEOS: bool = True,
    ):
        """
        Args
        - encoder_name: str, the name of the encoder, one of the keys of TRANSFORMER_INFO
        - embed_layer: int, the layer number of the embedding
        - iftrimCLS: bool, whether to trim the first classifification token
        - iftrimEOS: bool, whether to trim the end of sequence token, if exists
        """

        super().__init__(encoder_name, embed_layer)

        self._iftrimCLS = iftrimCLS
        self._iftrimEOS = iftrimEOS

        # load model from torch.hub
        print(f"Loading {self._encoder_name} using {self._embed_layer} layer embedding")
        self.model, self.alphabet = torch.hub.load(
            "facebookresearch/esm:main", model=self._encoder_name
        )
        self.batch_converter = self.alphabet.get_batch_converter()

        # set model to eval mode
        self.model.eval()
        self.model.to(DEVICE)

        self._embed_dim, self._max_emb_layer, _ = TRANSFORMER_INFO[self._encoder_name]

        assert (
            self._embed_layer <= self._max_emb_layer
        ), f"{self._embed_layer} exceeds {self._max_emb_layer}"

        expected_num_layers = int(self._encoder_name.split("_")[-3][1:])
        assert (
            expected_num_layers == self._max_emb_layer
        ), "Wrong ESM model name or layer"

    def _encode_batch(
        self,
        mut_seqs: Sequence[str] | str,
        flatten_emb: bool | str,
        mut_names: Sequence[str] | str | None = None,
    ) -> np.ndarray:
        """
        Encodes a batch of mutant sequences.

        Args:
        - mut_seqs: list of str or str, mutant sequences of the same length
        - flatten_emb: bool or str, if and how (one of ["max", "mean"]) to flatten the embedding
        - mut_names: list of str or str or None, mutant names

        Returns:
        - np.ndarray or a tuple(np.ndarray, list[str]) where the list is batch_labels
        """

        if isinstance(mut_names, str):
            mut_names = [mut_names]

        # pair the mut_names and mut_seqs
        if mut_names is not None:
            assert len(mut_names) == len(
                mut_seqs
            ), "mutant_name and mut_seqs different length"
            mut_seqs = [(n, m) for (n, m) in zip(mut_names, mut_seqs)]
        else:
            mut_seqs = [("", m) for m in mut_seqs]

        # convert raw mutant sequences to tokens
        batch_labels, _, batch_tokens = self.batch_converter(mut_seqs)
        batch_tokens = batch_tokens.to(DEVICE)

        # Turn off gradients and pass the batch through
        with torch.no_grad():
            # shape [batch_size, seq_len + pad, embed_dim]
            if batch_tokens.shape[1] > TRANSFORMER_MAX_SEQ_LEN:
                print(f"Sequence exceeds {TRANSFORMER_MAX_SEQ_LEN}, chopping the end")
                batch_tokens = batch_tokens[:, :TRANSFORMER_MAX_SEQ_LEN]

            encoded_mut_seqs = (
                self.model(batch_tokens, repr_layers=[self._embed_layer])[
                    "representations"
                ][self._embed_layer]
                .cpu()
                .numpy()
            )

        # https://github.com/facebookresearch/esm/blob/main/esm/data.py
        # from_architecture

        # trim off initial classification token [CLS]
        # both "ESM-1" and "ESM-1b" have prepend_bos = True
        if self._iftrimCLS and self._encoder_name.split("_")[0] in ["esm1", "esm1b"]:
            encoded_mut_seqs = encoded_mut_seqs[:, 1:, :]

        # trim off end-of-sequence token [EOS]
        # only "ESM-1b" has append_eos = True
        if self._iftrimEOS and self._encoder_name.split("_")[0] == "esm1b":
            encoded_mut_seqs = encoded_mut_seqs[:, :-1, :]

        if mut_names is not None:
            return self.flatten_encode(encoded_mut_seqs, flatten_emb), batch_labels
        else:
            return self.flatten_encode(encoded_mut_seqs, flatten_emb)


class CARPEncoder(AbstractEncoder):
    """
    Build a CARP encoder
    """

    def __init__(
        self,
        encoder_name: str,
        embed_layer: int,
    ):
        """
        Args
        - encoder_name: str, the name of the encoder, one of the keys of CARP_INFO
        - embed_layer: int, the layer number of the embedding
        """

        super().__init__(encoder_name, embed_layer)

        # load model from torch.hub
        print(f"Loading {self._encoder_name} using {self._embed_layer} layer embedding")

        self.model, self.collater = load_model_and_alphabet(self._encoder_name)

        self._embed_dim, self._max_emb_layer = CARP_INFO[self._encoder_name]

        assert (
            self._embed_layer <= self._max_emb_layer
        ), f"{self._embed_layer} exceeds {self._max_emb_layer}"

    def _encode_batch(
        self,
        mut_seqs: Sequence[str] | str,
        flatten_emb: bool | str,
        mut_names: Sequence[str] | str | None = None,
    ) -> np.ndarray:
        """
        Encodes a batch of mutant sequences.

        Args:
        - mut_seqs: list of str or str, mutant sequences of the same length
        - flatten_emb: bool or str, if and how (one of ["max", "mean"]) to flatten the embedding
        - mut_names: list of str or str or None, mutant names

        Returns:
        - np.ndarray or a tuple(np.ndarray, list[str]) where the list is batch_labels
        """

        mut_seqs = [[m] for m in mut_seqs]
        
        x = self.collater(mut_seqs)[0]

        layer_name = f"layer{str(self._embed_layer)}"

        activation = {}
        def get_activation(name):
            def hook(model, input, output):
                activation[name] = output.detach()

            return hook

        # convert raw mutant sequences to tokens
        self.model.model.embedder.layers[self._embed_layer].register_forward_hook(
            get_activation(layer_name)
        )

        rep = self.model(x)
        
        encoded_mut_seqs = activation[layer_name].numpy()

        return self.flatten_encode(encoded_mut_seqs, flatten_emb)

In [6]:
from scr.utils import pickle_load

In [7]:
df = pickle_load("data/proeng/gb1/two_vs_rest.pkl")

In [8]:
df_train = df.loc[(df["set"] == "train") & (df["validation"] != True)]
df_val = df.loc[(df["set"] == "train") & (df["validation"] == True)]
df_test = df.loc[(df["set"] == "test")]

len(df_train), len(df_val), len(df_test), len(df)

(381, 43, 8309, 8733)

In [43]:
no_flat_encoder = CARPEncoder(
    encoder_name="carp_640M",
    embed_layer=0,
).encode(mut_seqs=list(df_val.sequence))
one_emb = next(no_flat_encoder)
one_emb.shape

Loading carp_640M using 0 layer embedding


In [ ]:
mean_flat_encoder = CARPEncoder(
    encoder_name="carp_640M",
    embed_layer=0,
).encode(mut_seqs=list(df_val.sequence),flatten_emb="mean")
one_mean_emb = next(mean_flat_encoder)
one_mean_emb.shape